In [ ]:
# Please execute this cell (shift+<Return>) before starting the workbook
# this should print out "Your notebook is ready to go"
import sys
import tskit
import msprime

if "pyodide" in sys.modules: # if running in-browser (e.g. using JupyterLite)
    raise RuntimeError("Tsinfer/tsdate is not current supported when running in JupyterLite")

import genealogical_analysis_workshop

workbook = genealogical_analysis_workshop.setup_workbook3()
display(workbook.setup)

# Inference using `tsinfer` and `tsdate`

In the previous workbook we simulated genetic genealogies using `msprime`. Simulation is relatively simple compared to *inferring* genealogies from existing genetic data. Nevertheless, several software packages infer genetic genealogies which can be stored in tree sequence format. Here we will use a particularly scalable set of methods to do so: [tsinfer](https://github.com/tskit-dev/tsinfer) and [tsdate](https://github.com/tskit-dev/tsdate).

### A note about models vs reality

So far we have been using rather simplified models of population history. While slightly more realistic models [are available](https://elifesciences.org/articles/54967), it is still true that "All models are wrong, but some models are useful". In fact, when trying to infer the real history of interbreeding populations, the complexity of hybridization, migration, and mate choice probably justifies something more emphatic: "all models are wrong, but inferred population history models are especially wrong".

It is common to want to use genetic variation to establish or demolish support for particular models of population history, perhaps attempting to estimate parameters such as a migration rate. However, any cursory inspection of (say) human history should convince you that we shouldn't place much faith in a model of discrete populations with a handful of estimated parameters.

In this vein, when using the genealogical inference approaches below to look at real datasets, you are encouraged not to approach the analysis with a predefined model in your head. Rather you should use the results as an tool to explore the dominant patterns that emerge from the data. In particular, the `tsinfer` algorithm has relatively few underlying assumptions (e.g. it does not assume neutrality, fixed population sizes, or specific timings of ancestral events). That means it can be used to examine the data in a relatively model-free way.

In fact, *even if* your dataset has emerged from some specific model of population history, you still shouldn't expect your local trees to all look like some version of the known history. Even biologists investigating separate species shouldn't expect genomes to agree on a single "species tree", due to features such as incomplete lineage sorting and introgression. The best way of thinking of a species "phylogeny" is that it is a simplified version of the genetic genealogies encoded in the tree sequence.

Of course, if you are only interested in using tsinfer to compress your genome sequences, and don't care about how they came about, then this point is moot!

## Inferring topology using `tsinfer`

`Tsinfer` (pronounced t-s-infer) is comparable in some ways to other ancestral inference software such as [ARGweaver](https://doi.org/10.1371/journal.pgen.1004342), [Relate](https://myersgroup.github.io/relate/) (which we will encounter this afternoon), and [ARGneedle](https://palamaralab.github.io/software/argneedle/). However, it differs considerably in approach and scalability. Note that none of these other software packages produce their output in tree sequence (`tskit`) format, although is possible to convert their output to tree sequences. Also note that although `tsinfer` produces trees with a relatively accurate topology, but unlike other ancestral inference tools, it makes no attempt to produce branch length estimates -- for this we need another tool like `tsdate`.

The most important restrictions of `tsinfer` are:
1. It requires *phased* sample sequences
2. It only uses variants which have *known ancestral states* (other variants can be included, but do not contribute to the quality of inference)
3. It works better with full sequence data than with data from scattered target SNPs (e.g. as obtained from SNP chips).

Even though `tsinfer` is much faster than other comparable inference methods, inferences of large datasets (whole genomes from tens of thousands or samples or more) can take many days of computer time. For this reason, here we use a relatively small dataset that can be inferred in seconds. This is the genetic data that we simulated above using `msprime`. This comprises 5Mb of genome (roughly 5000 sites) from 160 individuals (320 diploid genomes), simulated under an 8 population stepping stone model. A benefit of using a simulated dataset is that we can estimate how good our inference is. As a reminder, here is a schematic of the model that was used to create the genetic data:

In [ ]:
import demes
import demesdraw

demesdraw.tubes(demes.load("data/simulated_8pop.yaml"));

### Input

`Tsinfer` requires a [SampleData](https://tsinfer.readthedocs.io/en/latest/api.html#sample-data) object as input. There are <a href="https://tsinfer.readthedocs.io/en/latest/tutorial.html#data-example">examples online</a> for how to create a SampleData file from the common VCF format. For speed, however, we have already created a SampleData file for genotypes from the simulation:

In [ ]:
import tsinfer

# With real genomic data you would probably create the .samples file from a VCF
sample_data = tsinfer.load("data/simulated_8pop.samples")

print(
    f"Data file has {sample_data.num_samples} haploid genome samples over",
    f"{sample_data.sequence_length/1e6:.2f} Mb with {sample_data.num_sites} variable sites"
)

Now that we have the genetic variation data in `SampleData` format (as phased haplotypes with known ancestral states), we can actually perform the inference.

The inference process is split into three independent steps (`generate_ancestors()`, `match_ancestors()` and `match_samples()`), but for small datasets there is a convenience function, [tsinfer.infer()](https://tsinfer.readthedocs.io/en/latest/api.html#tsinfer.infer) which runs all three steps at once. Inferring a tree sequence is thus as simple as calling this function:

<div class="alert alert-block alert-info"><b>Note:</b>
    For larger inferences, we recommend running <code>tsinfer</code> from the command-line rather than within the Python shell. To speed up inference, it is also possible to run it on a large number of computer processors in parallel.</div>

In [ ]:
# This may take a few tens of seconds
inferred_ts = tsinfer.infer(sample_data)
inferred_ts

And that's it! As you can see, the output is a `TreeSequence` object of the sort we have previously met, and which should be very efficient to process. In fact, this tree sequence is even slightly *smaller* than the simulated one. 

However, the inferred tree sequence is not a perfect reconstruction of the true (simulated) genealogy. To demonstrate, we'll print out a tree from the middle of the tree sequence, colouring sample nodes by their population:

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Create a style that colours nodes by their population, matching the demesdraw output
colour_map = plt.get_cmap("tab10")
hex_cols = [mpl.colors.rgb2hex(colour_map(p.id)) for p in inferred_ts.populations()]
plot_style = "".join([f".node.p{p.id} > .sym {{fill: {hex_cols[p.id]}}}" for p in inferred_ts.populations()])

middle_position = inferred_ts.sequence_length / 2
print(f"Tree at position {middle_position/1e6} Mb")
inferred_ts.at(middle_position).draw_svg(
    size=(1500, 400),  # Wider than the normal screen, to fit in all the samples
    node_labels={},  # Do not label nodes
    y_axis=True,
    y_ticks=np.arange(0, 1, 0.1),
    symbol_size=3,
    style=plot_style,
)

What you might notice:
1. There are some *polytomies* in the tree sequence (nodes with more than two descending branches), reflecting uncertainty in the tree
2. There are *unary nodes* in the tree (nodes with only one descending branch, appearing as points on straight branches). For technical reasons these can help to compress the data
3. The time scale runs from 0 to 1 rather than showing the number of generations

The last two points will be covered later, when we introduce `tsdate`.

<div class="alert alert-block alert-info">
<b>Note:</b>
A further minor point is that the first and last tree (before the first site and after the last site) are "empty": i.e. with no branches. This is because <code>tsinfer</code> does not try to extrapolate outside of the region for which it has data. In <code>tskit</code>, <a href="https://tskit.dev/tskit/docs/stable/data-model.html#sec-data-model-missing-data">missing data</a> is represented by samples which have no ancestry. It's also worth noting SampleData files can also cope with missing data (we'll see this later).
</div>

### How does `tsinfer` work?

<a href="https://www.nature.com/articles/s41588-019-0483-y/figures/2" style="float:right; margin: 1em;"><img src="pics/tsinfer.jpg"></a>`Tsinfer` is a heuristic algorithm, which means that it comes with no statistical guarantees of accuracy. However, as we shall see, it can be shown to do very well on simulated data, and has very few underlying assumptions. We will not go into much detail about how it actually works. Essentially, it combines a heuristic process for constructing likely looking ancestral DNA sequences (or "haplotypes") based on each variable site in turn, and a fast matching process for comparing these haplotypes to each other to determine which have inherited genetic material from which. If you want to find out more you can check out [the paper](https://doi.org/10.1038/s41588-019-0483-y) or [this explanatory video](https://www.youtube.com/watch?v=yB4uSle66Pw).

### Which sites are used for inference?

By default, the `tsinfer` algorithm assumes that at a site with an ancestral allele and a derived allele, the derived allele was the result of a single mutation some time in the past (i.e. each site has a single mutation).

<dl class="exercise"><dt>Exercise 1</dt>
<dd>Print out the first site in the inferred tree sequence.</dd></dl>

In [ ]:
# Exercise 1: Print out the first site (id=0) from the inferred_ts


In [ ]:
workbook.Q1()

Some sites can't be used for inference. In particular, this includes sites with more than two alleles (e.g. A, C, and T), and singletons (derived variants present in only a single sample). There are likely to be a number of these in our simulated dataset:

In [ ]:
print("Example of a site that was not used for inference")
for s in inferred_ts.sites():
    alleles = set([s.ancestral_state]) | set(m.derived_state for m in s.mutations)
    if len(alleles) > 2:
        print("-", s)
        print(f"  - Site {s.id} has alleles: {alleles}")
        break

Here you can see that the site at position 885059.0 has 3 allelic states. The site metadata has been used to flag this up as `"inference_type": "parsimony"`. That means that the mutations at this site were not used to construct the tree, but rather were placed on the tree later, using a [parsimony approach](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.Tree.map_mutations) (i.e. minimising the number of mutations needed). It is also possible to deliberately [exclude positions](https://tskit.dev/tsinfer/docs/stable/api.html#tsinfer.generate_ancestors) (e.g. for low-quality sites) such that mutations at those sites are also placed later, after inference has occurred.


### Real data

If you are running tsinfer on real datasets, rather than simulated ones, then chances are you will have to account both for sequencing error and missing data.

#### Missing data

Instead of specifing a known allele for a sample at a site, it is possible to input some data as <code>tskit.MISSING_DATA</code>. These will be <em>imputed</em> to the value suggested by the genealogy. Likewise, it is possible to set the ancestral state to <code>tskit.MISSING_DATA</code> and the site will be ignored for the purposes of inference, with the ancestral site imputed from the topology given by the surrounding sites. However, both of these advanced topics are outside the scope of this tutorial.

#### Sequencing error and multiple mutations

Real datasets usually have some component of sequencing error. If there is error but we assume that all observed variation is caused by a single mutation in the past, we'll place samples wrongly on the trees. To account for error, we treat errors as if they were an extra mutation at the same site. The ability to allow sites to have a small likelihood of having *multiple mutations* in `tsinfer` is called "mismatch", because it is implemented by allowing one ancestor to descend from another even if their patterns of variation don't exactly match. Note, however, that this capability is a relatively recent addition to tsinfer, and its effects have not been fully investigated.

To activate the mismatch capability, you need to specify a recombination rate (or you can pass an [msprime recombination RateMap](https://tskit.dev/msprime/docs/stable/api.html#msprime.RateMap)). Then you can tweak the relative contributions of recombination versus mismatch (which generates multiple mutations at a site) using the `mismatch_ratio` parameter. This defaults to a value of 1, which has been shown to give reasonable results in simulations. Knowing the mismatch ratio to set is somewhat of a dark art, and will depend on the expected error profile of your data, although simulations show that a large range of values from about 0.001 to 1000 give similar results.

Below we investigate the effect of allowing mismatch (and therefore creating fewer inferred trees and larger numbers of multiple mutations at some sites).

In [ ]:
import tskit

simulated_ts = tskit.load("data/simulated_8pop.trees") # the "ground truth" simulated ts
use_samples = [simulated_ts.samples(p.id)[0] for p in simulated_ts.populations()]
plot_region = (110_000, 120_000)  # only show a limited region of the genome
plot_params = dict(
    size=(1000, 200),
    x_lim=plot_region,
    time_scale="rank",
    mutation_labels={},
    node_labels={i:i for i in range(8)},
    style=plot_style,
)
prefix = f"{plot_region[0]//1000} kb - {plot_region[1]//1000} kb from the"
print(f"Plot (a) {prefix} true (simulated) tree sequence")
simulated_ts.simplify(use_samples).draw_svg(**plot_params)

In [ ]:
print(f"Plot (b): {prefix} inferred tree sequence (infinite sites / no error)")
inferred_ts.simplify(use_samples).draw_svg(**plot_params)

In [ ]:
print(f"Plot(c): {prefix} inferred tree sequence (allowing mismatch: fewer trees, some multiple mutations)")
# for speed only, use the first 1000 sites
short_sample_data = sample_data.subset(sites=np.arange(1000))
ts_default_mismatch = tsinfer.infer(short_sample_data, recombination_rate=1e-8)
ts_default_mismatch.simplify(use_samples).draw_svg(**plot_params)

In [ ]:
print("Plot (d) {prefix} inferred tree sequence (nearly all mismatch: many multiple mutations)\n may take a minute to run...")
ts_high_mismatch = tsinfer.infer(short_sample_data, recombination_rate=1e-8, mismatch_ratio=2e5)
ts_high_mismatch.simplify(use_samples).draw_svg(**plot_params)

Mutations are plotted both as crosses on the tree and stacked onto sites on the X axis. You can see that when the mismatch rate is set very high, we infer very few trees, with lots of mutations stacked onto the same sites (reversions or recurrent mutations). In this case, we know that this is incorrect: all the sites in this region had just a single mutation, as seen in plot (a).

In [ ]:
workbook.Q1bonus()

When there is sequencing error, soaking up the error into additional mutations (especially on terminal branches) is what we might want to do.

<dl class="exercise"><dt>Bonus thinking exercise</dt>
<dd>There are more trees in the original simulation than we have inferred: indeed one of the trees in plot (a) has no mutations at all, making it rather difficult to infer. However, look at the relationships in (b): do they mostly capture those in (a)?</dd></dl>

### Is the inference any good?

If you are using the inferred ts simply to calculate site-wise statistics, then it doesn't matter whether your inference is accurate:

<dl class="exercise"><dt>Exercise 2</dt>
<dd>Use the <code>.diversity()</code> method to print out the site-based genetic diversity of all your samples, both from the original tree sequence (<code>simulated_ts</code>) and the inferred one (<code>inferred_ts</code>). They should be the same.
<div class="alert alert-block alert-info"><b>Note:</b> You will not be able to run the branch length versions of the statistics on the inferred tree sequence, because although <code>tsinfer</code> provides a reasonable estimate of the <i>topology</i> of the tree sequence, it does not provide meaningful branch lengths. For that we need to <i>date</i> the tree sequence nodes. 
</div></dd></dl>

In [ ]:
# Exercise 2: Print out the site-based genetic diversity of the original and inferred tree sequences.


In [ ]:
workbook.Q2()

So if you only use site-base statistics, you can use an inferred tree sequence simply for computational efficiency.

Often, however, you want to treat the inferred genealogy as an noisy estimate of the true genealogy. This requires some way to measure the quality of inference. One [approach](https://doi.org/10.1093/genetics/iyac044) is to compare real and inferred ages of  mutations. However, this requires a *dated* tree sequence, which we will construct in the next section. A more qualitative, purely topological measure which does not require a dated tree sequence is the *genealogical nearest neighbour* statistic.

<div class="alert alert-block alert-info"><b>Note:</b>
Another topological approach is to use something like the <a href="https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.kc_distance">Kendall-Colijn tree distance metric</a> to compare topologies. However, it is not clear what values of this statistic should be compared against, especially when a tree has polytomies, so we will not cover it in this practical.
</div>

#### The genealogical nearest neighbour (GNN) metric

<img src="pics/GNN.png" style="float:left" width="250" height="250"> The GNN summarises features of the tree topologies without requiring knowledge of branch lengths. It requires us to split the samples into groups (here, the 8 simulated populations), and for each sample in turn (the "focal sample"), looks at the nearest neighbours in each local tree.

In the example to the left, if we take the leftmost (red) node as the focal sample, its nearest neighbours can be found by following the arrow up then down again. So the GNN proportions of the leftmost node are 50% red, 25% yellow, and 25% blue (i.e. `0.5 : 0.25 : 0.25`). Note that the proportions are not necessarily symmetrical: in this example, the average GNN from all reds to blue is quite small, but the GNN from blue to red is 50%.

In [ ]:
import seaborn as sns

# Compare the average GNN proportions from the true and inferred genealogies
gnn_vals = {}
for name, ts in [("true", simulated_ts), ("inferred", inferred_ts)]:
    gnn_vals[name] = np.array([
        np.mean( # take an average over all the samples in a population
            ts.genealogical_nearest_neighbours(
                ts.samples(i),
                [ts.samples(j) for j in range(inferred_ts.num_populations)]
            ),
            axis=0,
        )
        for i in range(inferred_ts.num_populations)
    ])

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4)) # Use plt.rcParams["figure.figsize"]
ax1.set_title("True GNN values")
sns.heatmap(gnn_vals["true"], vmin=0, vmax=0.5, ax=ax1)
ax2.set_title("Inferred GNN values")
sns.heatmap(gnn_vals["inferred"], vmin=0, vmax=0.5, ax=ax2);

You can see that the inference does extremely well at capturing the genealogical relationships. The diagonal shows that the closest relatives are indeed found in the same population, and the proportions of close relatives decrease as we move to successively more distant populations. We should have some confidence that the inference has worked.

Because these genealogical methods are so efficient, it is possible to calculate nearest neighbour analyses like this on huge datasets, such as the million genomes of the UK Biobank. An example set of figures is [here](https://www.nature.com/articles/s41588-019-0483-y/figures/5) (thumbnails below)

<a href="https://www.nature.com/articles/s41588-019-0483-y/figures/5"><img src="pics/UKBB-GNN.png"></a>

## Estimating tree sequence node times using tsdate

The `tsdate` program gives approximate Bayesian estimates for the dates of nodes (i.e. ancestral genomes) in the tree sequence. To do this it uses the number of mutations on edges of the tree sequence. This is important for dating demographic events, spotting effects of selection, and incorporate historical (aDNA) samples into inferred genealogies.

However, before we can date the inferred tree sequence, we must first preprocess it using `preprocess_ts()`. This method simplifies the tree sequence (so it does not contain "unary nodes", which are problematic to date) and removes long regions of no data, which can bias the results:

In [ ]:
# tsdate requires us to specify the mutation rate and a population
# size. In this case we know the mutation rate we used in the simulation
# but for real data this would need to be obtained separately
estimated_mutation_rate = 1e-8

# This may take a few tens of seconds
dated_ts = tsdate.date(
    tsdate.preprocess_ts(inferred_ts),
    mutation_rate=estimated_mutation_rate,
    Ne=4_000, # Effective population size: since our stepping-stone model has a lot of migration, so Ne ~=~ total pop size
    progress=True,
)

You can see that we need to know the historical mutation rate in our data, in unit time per base pair (where the time unit is generations by default). This scales the resulting dates to something meaningful. We also need a rough guess as to the "effective population size", which allows us to create a *prior* expectation for the ancestral node times, which (in the normal Bayesian way) gets updated by information from the mutations on the edges of the tree sequence. For more complex situations, it is possible to pass in alternative priors, but in general we might hope that the mutational data will overwhelm the prior, making it less important to get a precisely correct value of $N_e$.

Since we now have sensible branch lengths, it's now possible to calculate (for example) the mean branch length between samples (i.e. the branch-mode genetic diversity)

In [ ]:
dated_ts.diversity(mode="branch")  # Also try this after running tsdate.date above with another Ne, to see the effect of the prior

When we plot out the tree sequence, it now has a more sensible time-scale. We do this below, simplifying the dated tree sequence to aid plotting:

In [ ]:
subset_sample_ids = np.concatenate(
    [dated_ts.samples(p.id)[:4] for p in dated_ts.populations()] # Get four nodes from each population
)

simple_dated_ts = dated_ts.simplify(samples=subset_sample_ids)

simple_dated_ts.draw_svg(
    x_lim=[10_000, 30_000],
    size=(1_000, 400),
    node_labels={},
    mutation_labels={},
    y_axis=True,
    y_ticks=np.arange(0, 20_000, 5_000),
    symbol_size=3,
    style=plot_style
)

We can estimate how well the dating has done (which is also a reasonable test of the `tsinfer` step), by comparing times in the simulated tree sequence with times in the inferred tree sequence. However, unless the tree topologies are identical, the simulated and inferred tree sequences will not have directly comparable nodes. We could either exhaustively compare the time of every MRCA pairwise, or we could compare the node times below each mutation. Here's how to do the latter:

In [ ]:
def get_comparable_node_times(ts0, ts1):
    node_time_data = []
    for ts in (ts0, ts1):
        node_below_mutation_at_position = {}
        for s in ts.sites():
            if len(s.mutations) == 1:
                # Only save information for sites with a single mutation
                mutation_id = s.mutations[0].id
                node_id = ts.mutation(mutation_id).node
                node_time = ts.node(node_id).time
                if node_time > 0:
                    # Only use common ancestor nodes
                    node_below_mutation_at_position[s.position] = node_time
        node_time_data.append(node_below_mutation_at_position)
    time_data_ts0 = node_time_data[0]
    time_data_ts1 = node_time_data[1]
    shared_positions = set(time_data_ts0.keys()) & set(time_data_ts1.keys())
    return (
        np.array([time_data_ts0[pos] for pos in shared_positions]),
        np.array([time_data_ts1[pos] for pos in shared_positions]),
    )

true_node_times, inferred_node_times = get_comparable_node_times(simulated_ts, dated_ts)

cor_coef = np.corrcoef(np.log(true_node_times), np.log(inferred_node_times))
print("R squared of log times:", cor_coef[0,1]**2)

<dl class="exercise"><dt>Exercise 3</dt>
<dd><p>Use <code>plt.scatter(true_node_times, inferred_node_times, alpha=0.05)</code> to plot the inferred node times against the true node times, setting some alpha transparency on the plotted points.</p>

<p>Since variablility in time estimates is much greater for larger times, you should plot both X and Y on a log axis, using <code>plt.xscale('log')</code> and <code>plt.yscale('log')</code>. You can also label the axes using <code>plt.xlabel()</code> and <code>plt.ylabel()</code>, and add an $x=y$ line by <code>plt.plot([1, 1e4], [1, 1e4], "k-")</code>. How well did the dating work?</p></dd></dl>

In [ ]:
# Exercise 3: Plot true node times against inferred node times below the mutation at each site.


We can compare that with how we do when we cheat by using `tsdate` on the known ancestry, i.e. *redating* the ancestral nodes in the tree sequence. This should remove any errors introduced by tsinfer, leaving only errors caused `tsdate` relying on the number of mutations to estimate ancestor times ("branch lengths") in the trees:

In [ ]:
dated_true_topology_ts = tsdate.date(
    tsdate.preprocess_ts(simulated_ts),
    mutation_rate=estimated_mutation_rate,
    Ne=4_000, # Rough equivalent total effective population size in the stepping-stone model
)

true_node_times, reinferred_node_times = get_comparable_node_times(simulated_ts, dated_true_topology_ts)

plt.scatter(true_node_times, reinferred_node_times, alpha=0.05)
plt.xscale('log')
plt.yscale('log')
plt.plot([1, 10_000], [1, 10_000], "k-")
plt.show()

Unsurprisingly, the plot shows that we do a lot better at dating the times of ancestors when we know the true genealogy. The difference between this and the previous plot should give you some sort of feeling for how much noise is injected into the inference process by the errors in genealogical inference made by `tsinfer`.

## Analyses

<a href="https://awohns.github.io/unified_genealogy/interactive_figure.html"  style="float:right; margin: 1em" ><img src="pics/Unified-interactive.png"></a>If you have produced dated tree sequences from real genome data, we encourage you to use them for exploratory data analysis. For example, we can compare populations by looking at the distribution of times to the most recent common ancestor between samples picked from different populations. Although there is no time to go through the code in this workbook, you may be interested in exploring an [interactive online plot](https://awohns.github.io/unified_genealogy/interactive_figure.html) of human tMRCAs from our unified genealogy (thumbnail, right). Please ask your demonstrators to explain the plot if it isn't clear what is being shown, or to discuss interesting patterns that can be revealed.

More generally, once we have dates on our tree sequence, the entire bevvy of analytical results provided to us by `tskit` is now available. Some example of these are in the [extra workbook](./WorkshopExtra.ipynb), which you might like to investigate, if you have time.

## Some relevant papers and resources
 - The [tsinfer](https://tskit.dev/tsinfer/docs/stable/) and [tsdate](https://tsdate.readthedocs.io/en/latest/) documentation.
 -  Jerome Kelleher, Yan Wong, Anthony W Wohns, Chaimaa Fadil, Patrick K Albers and Gil McVean (2019) Inferring whole-genome histories in large population datasets. [Nature Genetics 51: 1330-1338](https://doi.org/10.1038/s41588-019-0483-y)
 -  Anthony Wilder Wohns, Yan Wong, Ben Jeffery, Ali Akbari, Swapan Mallick, Ron Pinhasi, Nick Patterson, David Reich, Jerome Kelleher, and Gil McVean (2022) A unified genealogy of modern and ancient genomes. [Science 375: eabi8264](https://www.science.org/doi/10.1126/science.abi8264)

## Acknowledgement
This workbook is heavily based on [Georgia Tsambos' Jupyter notebooks](https://github.com/gtsambos/2022-ts-workshops).